In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
from dateutil import parser
import pytz


# PART 1: PROCESS THE RACE FILE (TCX)

 def process_race_data(file_path):
    ns = {
        'ns': 'http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2',
        'ns3': 'http://www.garmin.com/xmlschemas/ActivityExtension/v2'
    }

    try:
        tree = ET.parse(file_path)
    except Exception as e:
        return f"Error loading race file: {e}"

    root = tree.getroot()
    data = []

    for trackpoint in root.findall('.//ns:Trackpoint', ns):
        tp = {}

        # Extract Time
        time_node = trackpoint.find('ns:Time', ns)
        if time_node is not None:
            tp['Time_UTC'] = parser.parse(time_node.text)

        # Extract Distance (Meters)
        dist_node = trackpoint.find('ns:DistanceMeters', ns)
        if dist_node is not None: tp['Distance_Meters'] = float(dist_node.text)

        # Extract Heart Rate
        hr_node = trackpoint.find('ns:HeartRateBpm/ns:Value', ns)
        if hr_node is not None: tp['HeartRate'] = int(hr_node.text)

        # Extract Altitude
        alt_node = trackpoint.find('ns:AltitudeMeters', ns)
        if alt_node is not None: tp['Altitude'] = float(alt_node.text)

        # Extract Speed (km/h from Coros)
        ext = trackpoint.find('ns:Extensions', ns)
        if ext is not None:
            for child in ext:
                if 'Speed' in child.tag:
                    try: tp['Speed_Raw'] = float(child.text)
                    except: pass
            if 'Speed_Raw' not in tp:
                 for child in ext:
                    if 'TPX' in child.tag:
                        for subchild in child:
                            if 'Speed' in subchild.tag:
                                try: tp['Speed_Raw'] = float(subchild.text)
                                except: pass

        if 'Time_UTC' in tp:
            data.append(tp)

    df = pd.DataFrame(data)

    # TIMEZONE CONVERSION
    if df['Time_UTC'].dt.tz is None:
        df['Time_UTC'] = df['Time_UTC'].dt.tz_localize('UTC')
    df['Time_IST'] = df['Time_UTC'].dt.tz_convert('Asia/Kolkata')

    # BASIC CLEANING
    df = df.dropna(subset=['HeartRate', 'Speed_Raw', 'Distance_Meters']).copy()

    # FEATURE ENGINEERING
    df['Distance_Miles'] = df['Distance_Meters'] * 0.000621371

    # Calculate Pace & Speed in MPH (Unit Fix)
    df = df[df['Speed_Raw'] > 0.5]
    df['Speed_MPH'] = df['Speed_Raw'] / 1.60934
    df['Pace_MinMile'] = 60 / df['Speed_MPH']

    return df

 # PART 2: THE 0.1 MILE BINNING LOGIC (FINAL)

 def aggregate_by_bin(df):
    # Create Bin ID (with Rounding Fix)
    df['Mile_Bin'] = ((np.floor(df['Distance_Miles'] * 10) / 10) + 0.1).round(1)

    # Group by this Bin
    df_binned = df.groupby('Mile_Bin').agg({
        'Time_IST': 'min',
        'Pace_MinMile': 'mean',
        'HeartRate': 'mean',
        'Altitude': ['first', 'last']
    }).reset_index()

    # Flatten Columns
    df_binned.columns = ['Mile_Bin', 'Time_IST', 'Avg_Pace', 'Avg_HR', 'Alt_Start', 'Alt_End']

    # Create Derived Metrics

    # Time Label (String Format)
    df_binned['Time_Label'] = df_binned['Time_IST'].dt.strftime('%H:%M:%S')

    # Elevation Change (Net)
    df_binned['Elevation_Change_m'] = df_binned['Alt_End'] - df_binned['Alt_Start']

    # Efficiency Factor (EF = Speed_MPH / HR)
    # We re-derive Speed_MPH from Avg_Pace to ensure alignment
    avg_speed_mph = 60 / df_binned['Avg_Pace']
    df_binned['Efficiency_Factor'] = avg_speed_mph / df_binned['Avg_HR']

    # Final Formatting (Rounding)
    df_binned['Avg_Pace'] = df_binned['Avg_Pace'].round(2)
    df_binned['Avg_HR'] = df_binned['Avg_HR'].round(0)
    df_binned['Elevation_Change_m'] = df_binned['Elevation_Change_m'].round(1)
    df_binned['Efficiency_Factor'] = df_binned['Efficiency_Factor'].round(4)

    # Column Cleanup & Reordering (As Requested)
    # Keep: Mile_Bin, Time_Label, Avg_Pace, Avg_HR, Elevation_Change_m, Efficiency_Factor
    final_cols = ['Mile_Bin', 'Time_Label', 'Avg_Pace', 'Avg_HR', 'Elevation_Change_m', 'Efficiency_Factor']

    return df_binned[final_cols]

 # PART 3: PROCESS READINESS DATA

 def process_readiness_data(file_path):
    df = pd.read_csv(content/readiness - Sheet1.csv)
    df['Date'] = pd.to_datetime(df['Date'])

    def parse_sleep_string(time_str):
        try:
            parts = str(time_str).split(':')
            return round(int(parts[0]) + (int(parts[1]) / 60), 2)
        except:
            return None

    df['Sleep_Hours_Decimal'] = df['Sleep_Duration'].apply(parse_sleep_string)
    return df

# EXECUTION

# Race Data
df_raw = process_race_data('473808275823951980.tcx')
if isinstance(df_raw, pd.DataFrame):
    df_final = aggregate_by_bin(df_raw)

    df_final.to_csv('marathon_binned_final.csv', index=False)
    print(f"✅ Race Data Processed! (Rows: {len(df_final)})")
    print("📁 Saved as: 'marathon_binned_final.csv'")
    print(df_final.head().to_markdown(index=False))

# Readiness Data
try:
    df_health = process_readiness_data('readiness - Sheet1.csv')
    df_health.to_csv('readiness_final.csv', index=False)
    print("\n✅ Readiness Data Processed!")
except Exception as e:
    print(f"⚠️ Readiness data waiting: {e}")

✅ Race Data Processed! (Rows: 265)
📁 Saved as: 'marathon_binned_final.csv'
|   Mile_Bin | Time_Label   |   Avg_Pace |   Avg_HR |   Elevation_Change_m |   Efficiency_Factor |
|-----------:|:-------------|-----------:|---------:|---------------------:|--------------------:|
|        0.1 | 03:00:19     |      11.95 |      123 |                    0 |              0.0409 |
|        0.2 | 03:01:17     |       9.36 |      145 |                   -2 |              0.0443 |
|        0.3 | 03:02:13     |       8.87 |      150 |                   -1 |              0.0451 |
|        0.4 | 03:03:04     |       8.31 |      155 |                   -1 |              0.0466 |
|        0.5 | 03:03:53     |       8.28 |      156 |                   -1 |              0.0465 |
⚠️ Readiness data waiting: name 'content' is not defined
